# Ancillary external photometry

This tutorial shows how to use external photometric data to perform astrometry calibrations on a Cube

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

from pykoala.data_container import Cube
from pykoala.instruments.koala_ifu import koala_rss

rss = koala_rss("data/27feb20035red.fits")
rss.intensity /= rss.info["exptime"]
rss.variance /= rss.info["exptime"]**2

In [ ]:
from pykoala.corrections.flux_calibration import FluxCalibration

fcal = FluxCalibration.from_text_file('products/response_HILT600_transfer_function.dat')

rss = fcal.apply(rss)

In [ ]:
plt.figure()
plt.plot(rss.wavelength, rss.snr[500])

In [ ]:
from pykoala.photometry import query_image, PSQuery

# Query r-band photometry from PS using a cutout with the size of the input cube
# + 30 additional arcseconds on every side. Save the fits in the current working directory
# image_query_results = query_image([cube], filters='i', im_extra_size_arcsec=30, im_output_dir='.')

image_query_results = {"PS1.i": {}}
image_query_results["PS1.i"]["ccd"] = PSQuery.read_ps_fits("ps_query_129.0629_n26.4126_i.fits")
image_query_results["PS1.i"]["pix_size"] = PSQuery.pixelsize_arcsec

In [ ]:
from pykoala.corrections.astrometry import AstrometryOffsetCorrection

astrometric_correction, results = AstrometryOffsetCorrection.from_external_image(
    rss, image_query_results["PS1.i"], filter_name="PANSTARRS_PS1.i")



In [ ]:
results["offset_fig"]